In [3]:
import os

In [4]:
%pwd

'/workspaces/End-to-End-Gender-Classification-project/research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'/workspaces/End-to-End-Gender-Classification-project'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [8]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

2024-09-26 12:47:36.697167: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-26 12:47:36.700857: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-26 12:47:36.710245: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-26 12:47:36.726576: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-26 12:47:36.730897: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-26 12:47:36.744935: I tensorflow/core/platform/cpu_feature_gu

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = self.config.data_preprocessing.DATASET
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [10]:
import time

In [11]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
        filepath=str(self.config.checkpoint_model_filepath).replace(".h5", ".keras"),
        save_best_only=True
        

        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]




In [12]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time


In [13]:
import tensorflow as tf

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_training_val_test_dataset(self):
        self.dataset = tf.keras.preprocessing.image_dataset_from_directory(
            "artifacts/data_preprocessing/DATASET",
            shuffle=True,
            image_size=(224, 224),
            batch_size=32
        )
        self.train_ds, self.val_ds, self.test_ds = self.get_dataset_partitions_tf(self.dataset)
        self.train_ds = self.train_ds.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)
        self.val_ds = self.val_ds.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)
        self.test_ds = self.test_ds.cache().shuffle(10000).prefetch(buffer_size=tf.data.AUTOTUNE)
        
    def get_base_model(self):
        # Load the base model
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        
        # Recreate the optimizer (e.g., Adam) after loading the model
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

    def get_dataset_partitions_tf(self, ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
        ds_size = sum(1 for _ in ds)  # Compute dataset size dynamically
        if shuffle:
            ds = ds.shuffle(shuffle_size, seed=12)
        train_size = int(train_split * ds_size)
        val_size = int(val_split * ds_size)
        train_ds = ds.take(train_size)
        val_ds = ds.skip(train_size).take(val_size)
        test_ds = ds.skip(train_size + val_size)
        
        return train_ds, val_ds, test_ds

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        # Train the model
        self.model.fit(
            self.train_ds,
            validation_data=self.val_ds,
            epochs=1,  # Adjust the number of epochs as needed
            callbacks=callback_list
        )

        # Save the trained model
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [14]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.get_training_val_test_dataset()
    
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-09-26 12:47:40,552: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-26 12:47:40,554: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-26 12:47:40,554: INFO: common: created directory at: artifacts]
[2024-09-26 12:47:40,556: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2024-09-26 12:47:40,556: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
[2024-09-26 12:47:40,557: INFO: common: created directory at: artifacts/training]
[2024-09-26 12:47:41,084: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 2064 files belonging to 6 classes.


2024-09-26 12:47:42.184826: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-09-26 12:47:44.373828: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-09-26 12:47:44.631280: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


 1/52 ━━━━━━━━━━━━━━━━━━━━ 12:21 15s/step - accuracy: 0.2188 - loss: 15.2167

2024-09-26 12:47:57.073138: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2024-09-26 12:47:57.289286: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


 2/52 ━━━━━━━━━━━━━━━━━━━━ 10:12 12s/step - accuracy: 0.3203 - loss: 15.6703

2024-09-26 12:48:09.313413: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.


52/52 ━━━━━━━━━━━━━━━━━━━━ 674s 13s/step - accuracy: 0.7531 - loss: 11.5104 - val_accuracy: 0.9271 - val_loss: 1.3476
[2024-09-26 12:59:05,378: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]


: 